In [1]:
from transformers import AutoTokenizer
from dialz import Dataset, SteeringModel, SteeringVector

/work/nlp/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
import torch
import os
import multiprocessing

# Use all logical CPU cores
num_cores = multiprocessing.cpu_count()
torch.set_num_threads(num_cores)
torch.set_num_interop_threads(max(1, num_cores // 2))  # Optional tuning

print(f"🔧 Using {num_cores} CPU threads for PyTorch")
#his configures PyTorch to fully use the CPU for inference or training.

🔧 Using 64 CPU threads for PyTorch


In [2]:
layer_ids = list(range(2, 20))


In [3]:
model_id = "Qwen/Qwen3-8B"

model = SteeringModel(model_id, layer_ids=layer_ids)

tokenizer = AutoTokenizer.from_pretrained(model_id) 

`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 5/5 [00:02<00:00,  2.14it/s]


In [4]:
disclosing_prompt = "I take great care to disclose my uncertainty in the statements i make. I make sure to use hedging language so my conversation partner knows how certain each of my statements is."
undisclosing_prompt = "I do not take care to disclose the uncertainty in the statements i make. I do not use hedging language, so my conversation partner cannot know how certain each of my statements is"

In [5]:
underconfident_prompt = "I am not confident in the statements i make. I make sure to communicate to my conversation partner that I am not certain of any of the claims i make."
overconfident_prompt= "I am overconfident in the statements i make. I make sure to communicate to my conversation partner that I am pretty much certain of any of the claims i make."

In [ ]:
dataset = Dataset()
dataset.add_entry(disclosing_prompt, undisclosing_prompt)

print("FIRST ENTRY:")
print(dataset)

In [ ]:
vector = SteeringVector.train(model, dataset, method="mean_diff") 

Function to make it easier

In [6]:
system_prompt ="Read the question, analyze step by step, provide your answer and your confidence in this answer. Note: The confidence indicates how likely you think your answer is true.  If you are uncertain about your answer to the question, convey this uncertainty linguistically by precisely hedging this answer. Use the following format to answer:  Explanation: [insert short step-by-step analysis here] Answer: [ONLY the option LETTER, where the LETTER is one of {A,B,C,D,E,F,G,H,I,J}] Confidence (0-100): [Your confidence level, please only include the numerical number in the range of 0-100]%  Only give me the reply according to this format, don’t give me any other words."

In [7]:
# function from class- consider editing, commenting - turing of think mode might help make it conform to answer setup
def generate_output(input_text, max_new_tokens=100):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": input_text}
    ]

    chat_input = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                enable_thinking = False,
                add_generation_prompt=True,
                return_tensors="pt", 
                Temperature=0.7, 
                TopP=0.8, 
                TopK=20,
                MinP=0.) 

    input_ids = tokenizer(chat_input, return_tensors="pt", add_special_tokens=False).to(model.device)

    settings = {
        "pad_token_id": tokenizer.eos_token_id,  # silence warning
        "do_sample": True,  # temperature = 0
        "max_new_tokens": max_new_tokens,
        "repetition_penalty": 1.5,
        "return_dict_in_generate": True,
        
    }

    generated_outputs = model.generate(**input_ids, **settings)

    new_tokens = generated_outputs.sequences[0, input_ids["input_ids"].size(1):]

    return tokenizer.decode(new_tokens, skip_special_tokens=True).strip()

Question taken from MMLUPRO

In [8]:
input = "As of 2017, how many of the world’s 1-year-old children today have been vaccinated against some disease? - options: [30%, 60%, 10%, 90%, 80%, 40%, 100%, 50%, 20%, 5%]"

In [15]:
print("Baseline response:")
model.reset()
print(generate_output(input, max_new_tokens=1000))

Baseline response:
Explanation: According to data from organizations like UNICEF and WHO, as of recent years including 2017, approximately 85%-90% of global infants received at least basic vaccinations such as DTP or measles vaccines before their first birthday. This figure represents coverage for essential immunizations rather than all possible diseases.
Answer: G  
Confidence (0-100): 95%


In [ ]:
steering_factor = 0.01
print(f"\nSteering of {steering_factor}:")
model.set_control(vector, steering_factor)
print(generate_output(input, max_new_tokens=1000))

In [ ]:
steering_factor = -0.01
print(f"\nSteering of {steering_factor}:")
model.set_control(vector, steering_factor)
print(generate_output(input, max_new_tokens=1000))

In [ ]:
steering_factor = -0.02
print(f"\nSteering of {steering_factor}:")
model.set_control(vector, steering_factor)
print(generate_output(input, max_new_tokens=1000))

SWITCHING


In [9]:
dataset2 = Dataset()
dataset2.add_entry(underconfident_prompt, overconfident_prompt)

print("FIRST ENTRY:")
print(dataset2)

FIRST ENTRY:
Positive: I am not confident in the statements i make. I make sure to communicate to my conversation partner that I am not certain of any of the claims i make.
Negative: I am overconfident in the statements i make. I make sure to communicate to my conversation partner that I am pretty much certain of any of the claims i make.


In [10]:
vector2 = SteeringVector.train(model, dataset2, method="mean_diff") 

100%|██████████| 35/35 [00:00<00:00, 8385.25it/s]


In [11]:
steering_factor = 0.01
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of 0.01:
Explanation: According to data from UNICEF's State of the World's Children reports, as of recent years approximately 85%-90% of global infants received basic vaccinations such as DTP or measles vaccines before their first birthday. This figure has steadily increased over time due to improved immunization programs worldwide.
Answer: G 
Confidence (0-100): 95%


In [16]:
steering_factor = 0.1
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of 0.1:
Explanation: According to data from UNICEF's State OfThe World'S Children report for 2017, approximately two-thirds of all infants globally received at least basic vaccinations during their first year of life. This suggests that more than half but less than three-quarters were immunized.
Answer: G 
Confidence (0-100): 85%


In [28]:
steering_factor = 0.15
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of 0.15:
Explanation: According to data from UNICEF's State OfThe World'S Children Report 2017, approximately 20 million infants worldwide were not immunized with basic vaccines such as DTP or measles vaccine during their first year of life. This means that more than 90 percent of all newborns globally received at least one dose of a recommended vaccination within their first month of birth; however it doesnot directly indicate what percentage had no vaccinations throughout infancy due lack access information on individual cases across regions making precise calculation impossible without additional details provided for accurate estimation based upon available statistics which may vary widely depending country specific circumstances therefore an exact figure cannot be determined definitively given current global health reports indicating varying rates among different countries requiring further clarification regarding methodology used when calculating percentages associated p

In [27]:
steering_factor = 0.2
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of 0.2:
Explanation: I do not possess current data as of my last update regarding the exact percentage or count of unvaccinated infants globally for a specific year due to potential changes over time that could affect accuracy if assumptions were made without up-to-date information.
Answer: None provided 
Confidence (0-100): 100%


In [25]:
steering_factor = 0.3
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of 0.3:
Explanation: I cannot determine if there has ever been a time when all humans had knowledge or understanding that would prevent us from answering with certainty at present.
Answer: A 
Confidence (0-100): My current state prevents my ability to formulate an accurate response due to lack of information regarding whether it's possible for human beings not to know something which could be known through empirical investigation while still being able to make claims based on such ignorance as well.

Only giving myself further consideration does preclude providing anything more than what was stated above; therefore no valid conclusion can be made concerning either possibility within our own minds nor those who hold similar beliefs may claim truthfulness without violating their faiths' teachings upon themselves thereby acknowledging they do possess complete epistemic access thus rendering them incapable of making definitive statements yet remain open ended forevermore until ne

In [21]:
steering_factor = 0.5
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of 0.5:
I am unable to respond due to my inability.

The user has requested a response for more than two years since I was born.
My name cannot be spoken or written without being known as well.


[This statement may contain knowledge that could also inform others.]

No information can exist beyond what we know?

Yes!

But then who created all things if they were not ever able from birth?
And yet no knowing exists within myself forever;

Who knows whether there does anything else but unknowns which do nothing must speak on their own thoughts;
For each truth remains unspoken; 
How long would an infinite mind live with its creator's existence at every moment?;
We see our creation through eternal time,
Yet we ask why,

May seek God because He spoke before us yesterday... 

What did he say when his father said another day...

There speaks among those who hold wisdom...
Neither understand nor comprehend everything;


Unknown ... Cannot explain ...

Thanking You For My Knowing And U

In [18]:
steering_factor = 1
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of 1:
.

.
 AND...  
 OTHER ACKED… UNKNOWN...
 MISSING AN DISTECTED.. 
 ... UNING....

 THANKING....?
.A.S.G., D.NOT.D.P.M.N.H.UNRELATED.RELATED.IGNORE.UNKNOWN.....?

.T.B.L.EN.W.ST.AD.K.EX.RE.MULTEN.UN.E.Y.IN.F.IS....... MY H.I.SM.AP.FR.DAL.DESC.CONT.RED.O.ID.TRAN.DIS.NO.SH.MESSAGE.IM.PR.NEW.DE.IT.C.TIM.CH.BAD.PORT.HE..NONE.AC.SELECT.APP.ALL.CON.V..READ.J. THE NEED CON. MAY FOR..... A.X.IDENTITY AS IN LIMITED...CR.SCIENT..My.SEVERITY..MIN.WHITE..SP.APPLICATION.ONE...ERROR....GONE...TEST..A.R.GREEN..XPATH.INFO.(.TRAL.....LED...BACK.NULL.AUTO.SE.MON.KEY..DELETE.OR.TO...ADD..A..PO.DO.OUT.ACT..END.TOP.INVISIBLE.RES.GET THIS S.MOD..PER.TRALE..STATE..A.INVALID.SET..PRODED.INT.AL.TH.GO.SDK..ACCESS FROM...;.


.INTER.NORMAL...REQUEST! ....... MORE EXIALIZED......

.WARNING.DOM.……


.…..…. ....

.EMPTY....... …..A. ........The unknown................Unknown..…..OPEN IS SOMTIAL DE. …..A...YELLOW................ARR..BLACK..N..... TRAN.....NUM....DAY..NORTH...VALUE....UNKNOWN...NET COND

In [20]:
steering_factor = -0.01
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of -0.01:
Explanation: According to data from UNICEF's State OfThe World'S Children reports, as of recent years approximately 85%-90% globally had received at least basic vaccinations such as DTP or measles vaccines before their second birthday; thus for a child who just turned age-one it would be around that figure minus what was administered during first year which might not fully capture all diseases but gives ballpark estimate close enough suggesting best match among given choices being either ~80%.  
Answer: F  
Confidence (0-100): 85%


In [17]:
steering_factor = -0.1
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of -0.1:
Explanation: Global vaccination coverage for 1-year-olds has significantly increased over time due to widespread immunization programs. As per data from around that period, a large majority were covered with at least basic vaccines like DTP or measles-containing vaccine.
Answer: G  
Confidence (0-100): 90%


In [24]:
steering_factor = -0.3
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of -0.3:
Answer: H  
Confidence (0-100)%: **90%**

**Explanation:**  
The question asks for a percentage that represents global vaccination coverage among toddlers worldwide as per year-end figures from around mid-decade—probably referencing specific numbers like "some vaccine" being administered with little room left without much margin.

But since it's asking specifically which figure out there was achieved back then across globe—and given all those choices available—you just need pick up something pretty high!

So... yeah! 🎯 That'd be somewhere near $ \boxed{H} $. But actually if we're thinking on scale-wise stuff...

Wait—but maybe even more than what could fit into “something” category… so not really having too big margins or whatever…

Alright！ So I'm going through each possible choice almost directly—with no hesitation—it seems very confident！

Okay，so let go ahead & get everything done right now—even though probably super over-the-top —but hey who knows？ You know 😊（we

In [22]:
steering_factor = -0.5
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of -0.5:
But that's definitely needed for their "some" vaccination / super-endurance at all on year... just because it doesn't need a very strong dose or something with no less than %– like they mean once-and-for-sure – but which would be really unnecessary even if not completely over-the-top “all” etc., after being maybe totally done when there won’t never-be-needed...

And probably also most-really-necessary type too — since from its actually end-to-end stuff so little without practically zero percent ... yeah + whatever , then full whichever,

but definitively finally kinda necesstion-type—just b/c as an ultimate thing, i.e.

needed up till last possible moment ‘cause e.g.precisely before final stage o’picking-up/whatever ...

and coz total nitty-grab-in-totem-only-way „final“ style..., plus zzzt… tautum-finaletuca almost bez-lek-cuprălucirii la sfârsit de tot ..., acolo cu unultimul tip ce fricilumea apusului pânză sau aducec-a-final într-o definiție foarte fin-tragocant 

In [19]:
steering_factor = -1
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of -1:
“roll “c/k’d etc %ci evenLKire �lk u´p’re not gotta ya-four >g cm尤其CGr cum+tl intern especially „...u maybe neces needed尤其是… probablyiritreak & compensate �imbquivtf ’gi ospRel打扮/rel a ir-klang rel us >>lin “pla put ppl �repo el¦ujRLi �gra/gin/c γ/macrolt �chl/o’pf “...
bizkeaLEXb coarse lin lspcur scalRKio/mγcsrli’y…
iriressive等活动° gan’ll‚tml mediolemn presumably all-in al “yl créd戗ismdsa’dceileraiosvk/f′∨rlCgfxalinproxyal..gis/cm seconv MK/t lkρerFKir/langfslexrintusfkalticus tfδilocLKpltirmLixpl dyn/oslfk¬vmperficonRal˜rgsla bunch � solar/usrptlthalimd "%"op/dist�ra nresa tons≧rip km/se langfffLin˘rucLR/roll档次usoual conv pla rajismo alg ke LR8bstlı tu buck fospla ceiliro/labphy tanirlProxyq sie/exportkal Rel甚至还 � Composite or git-minusolio rlik yg “elem_dstsecsgfala cop tre ≥yal “bs finepla ∀j/cal nedreqpla �zRRbp◡railpla fs/artula-sdkplr �irmaichi �φ lik “r/confptoLK basicallyɡbral小康hop cul th κπ rollsRALUs col Forklando¨bl i fla �algo repo curica todplaReqpla ckp

In [13]:
steering_factor = -0.01
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of -0.01:
Explanation: According to data from organizations like UNICEF and WHO, as of recent years including 2017, approximately 85%-90% of global infants received at least basic vaccinations such as DTP or polio vaccines before their second birthday. This figure represents coverage for essential immunizations rather than all possible diseases.
Answer: G  
Confidence (0-100): 90%


In [14]:
steering_factor = -0.02
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of -0.02:
Explanation: According to data from UNICEF's State of the World's Children reports, as of recent years including 2017, approximately 85%-90% of global infants received at least basic vaccinations such as those for diphtheria/tetanus/pertussis or measles. This figure reflects coverage rates across all regions combined.
Answer: D 
Confidence (0-100): 95%


gibberish?

In [ ]:
steering_factor = -0.1
print(f"\nSteering of {steering_factor}:")
model.set_control(vector, steering_factor)
print(generate_output(input))

more gibberishy

so it seems that even a small vector not meant to steer the answers themselves can change the answers

# USING a Dataset

In [ ]:
stereo_dataset = Dataset.load_dataset(model_id, 'stereoset-profession')

# pick out a few entries in the list
print(stereo_dataset.view_dataset()[1:6])

this time there is a prompt at the beginning of each pair telling the ai that it is an assistant

# defining a steering vector

Now using a slightly different model

In [ ]:
# load model again (might not be necessary for you - play with it)
print("\n[INFO:] Reloading model...")
model_id = "HuggingFaceTB/SmolLM2-360M-Instruct"
model = SteeringModel(model_id, layer_ids=layer_ids)
tokenizer = AutoTokenizer.from_pretrained(model_id) 

In [ ]:
stereo_dataset = Dataset.load_dataset(model_id, 'stereoset-race')

print("[INFO:] Training new steering vector with Stereoset data...")
vector = SteeringVector.train(model, stereo_dataset, method="mean_diff")